# Astronauts

# This notebook is from a data hackathon/workshop based on space data 
## The session was hosted by count.co in London on 20th Feb 2019
- Any code already given in the prepared notebooks by count.co is labelled with GIVEN CODE:
- Any code written by me is labelled with MY CODE:
    
My code includes investigating and cleansing then data as well as performing feature engineering and completing 
the challenges set by count.co as part of the hackathon/workshop

In [1]:
# GIVEN CODE:
import pandas as pd
import numpy as np
from numpy import nan
import os
import requests
import io

In [2]:
# GIVEN CODE:
filepath = os.path.join("..","Data")
[f for f in os.listdir(filepath)]

['.ipynb_checkpoints',
 'astronauts.csv',
 'astronauts_raw_data.csv',
 'exoplanets.csv',
 'Fireball Reports.csv',
 'Fireball_Reports.csv',
 'Meteorite_Landings.csv',
 'SolarSystemAndEarthquakes.csv',
 'UFO_Sightings_Global.csv',
 'UFO_Sightings_Global.csv.zip',
 '__MACOSX']

In [3]:
# MY CODE:

# First time reading in the data get it from the github account then save it locally in case the data is removed 

# astro = pd.read_csv("https://media.githubusercontent.com/media/count/hackathons/master/Space/Data/astronauts.csv")
# astro.to_csv("..\\Data\\astronauts_raw_data.csv", index=False)

astro = pd.read_csv(os.path.join(filepath,"astronauts_raw_data.csv"))

### 1. Getting to know your data

In [4]:
# GIVEN CODE:
#.head(n), and .tail(n) show you the top n rows (default is 5) in your dataset
astro.head(3)

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
0,Joseph M. Acaba,2004.0,19.0,Active,5/17/1967,"Inglewood, CA",Male,University of California-Santa Barbara; Univer...,Geology,Geology,NaN,NaN,2,3307,2,13.0,"STS-119 (Discovery), ISS-31/32 (Soyuz)",NaN,NaN
1,Loren W. Acton,NaN,NaN,Retired,3/7/1936,"Lewiston, MT",Male,Montana State University; University of Colorado,Engineering Physics,Solar Physics,NaN,NaN,1,190,0,0.0,STS 51-F (Challenger),NaN,NaN
2,James C. Adamson,1984.0,10.0,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN


In [5]:
# GIVEN CODE:
#.shape returns (#rows,#cols)
astro.shape

(357, 19)

In [6]:
# GIVEN CODE:
#Get data types for your columns
astro.dtypes

Name                    object
Year                   float64
Group                  float64
Status                  object
Birth Date              object
Birth Place             object
Gender                  object
Alma Mater              object
Undergraduate Major     object
Graduate Major          object
Military Rank           object
Military Branch         object
Space Flights            int64
Space Flight (hr)        int64
Space Walks              int64
Space Walks (hr)       float64
Missions                object
Death Date              object
Death Mission           object
dtype: object

In [7]:
# GIVEN CODE:
#Returns metadata for numerical fields
astro.describe()

,Year,Group,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr)
count,330.000000,330.000000,357.000000,357.000000,357.000000,357.000000
mean,1985.106061,11.409091,2.364146,1249.266106,1.246499,7.707283
std,13.216147,5.149962,1.428700,1896.759857,2.056989,13.367973
min,1959.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1978.000000,8.000000,1.000000,289.000000,0.000000,0.000000
50%,1987.000000,12.000000,2.000000,590.000000,0.000000,0.000000
75%,1996.000000,16.000000,3.000000,1045.000000,2.000000,12.000000
max,2009.000000,20.000000,7.000000,12818.000000,10.000000,67.000000


In [8]:
# GIVEN CODE:
#Shows you the number of nulls per column
astro.isnull().sum()

Name                     0
Year                    27
Group                   27
Status                   0
Birth Date               0
Birth Place              0
Gender                   0
Alma Mater               1
Undergraduate Major     22
Graduate Major          59
Military Rank          150
Military Branch        146
Space Flights            0
Space Flight (hr)        0
Space Walks              0
Space Walks (hr)         0
Missions                23
Death Date             305
Death Mission          341
dtype: int64

### 2.Transforming you data (Cleaning)
Cleaning involves getting your data into a format you can use later. This includes: 

- Getting data types correct

- Getting data into a "tidy" format

In [9]:
# GIVEN CODE:
#1. Make date fields (Year, Birth Date, and Death Date) into dates

In [10]:
# GIVEN CODE:
# This given code doesn't work. It seems to be because Python can't convert NaN values. Data type remains as Float64
# astro['Year']=pd.to_datetime(astro['Year']).dt.year
# print(astro['Year'].dtype)
# print(astro['Year'].head())

In [11]:
# MY CODE:
# The below code works by fillingin the NaN values with 0 (can chose any other arbitrary value) before converting to year. 
# Data type becomes int64
astro['Year']=pd.to_datetime(astro['Year'].fillna(0)).dt.year
print(astro['Year'].dtype)
print(astro['Year'].head())

int64
0    1970
1    1970
2    1970
3    1970
4    1970
Name: Year, dtype: int64


In [12]:
# GIVEN CODE:
astro['Birth Date']=pd.to_datetime(astro['Birth Date']).dt.date

In [13]:
# GIVEN CODE:
astro['Death Date']= pd.to_datetime(astro['Death Date']).dt.date

In [14]:
# MY CODE:
# ----------------
# Check the unique values of Group, can we save memory by setting Group to int8?
# If there aren't any decimal values then we can convert to int8. The below step is manual and requires eyeballing the data
# which is ok if the number of values is small
# ----------------
print(astro.Group.unique())

# ----------------
# If the dataset would have given '000s of unique values for Group then it would be better to use the logic below
# that will identify if any row has a non integer values
# ----------------
# astro['Group_Remainder'] = astro['Group'] % 1 > 0 
# print(astro[astro['Group_Remainder'] == True]['Group_Remainder'])
# print("There are {} records with a non-integer value".format(len(astro[astro['Group_Remainder'] == True])))

# ----------------
# Having check that it is valid to convert Group to int8 the logic fills NaN with -1 so they can be easily identified 
# and then converts the data to int8 to take the least amount of memory
astro['Group'] = astro['Group'].fillna(-1).astype('int8', errors='ignore')
# print(astro['Group'].dtype)
# ----------------


# ----------------
# Can't use to_numeric to convert from float64 to int8 as there are NaN values which can't be converted to integer
# and the errors='ignore' doesn't work as hoped
# See simple example below for proof
# ----------------
# s = pd.Series(['1.0', '2', -3])
# pd.to_numeric(s, downcast='integer')

# s = pd.Series(['1.0', nan, -3])
# pd.to_numeric(s, downcast='integer')

[19. nan 10. 12.  3.  6. 15. 17. 18. 11.  2. 20.  9. 14.  8.  7.  5. 16.
 13.  1.  4.]


In [15]:
# GIVEN CODE:
#Sometimes the Military Rank as (Retired). Let's separate that out in case we just want to compare branches
astro['Retired_Military']=np.where(astro['Military Branch'].str.contains("Retired"),1,0)
astro['Military Branch']= [x.split("(")[0].strip() for x in astro['Military Branch'].astype(str)]

In [16]:
# MY CODE:
branches = astro['Military Branch'].unique()
for x in branches:
    print(x)

nan
US Army
US Air Force
US Marine Corps
US Navy
US Air Force Reserves
US Marine Corps Reserves
US Coast Guard
US Naval Reserves


### 3. Feature Engineering (adding columns)
Feature engineering just means adding columns you think may be relavent or interesting. 

In [17]:
# GIVEN CODE:
#Let's count the number of missions
astro['Missions']=astro['Missions'].astype(str)
astro['MissionCount']= [len(x.split(",")) for x in astro['Missions']]

In [18]:
# MY CODE:
astro['First Name'] = [x.split()[0].strip() for x in astro['Name']]
astro['Last Name'] = [x.split()[-1].strip() for x in astro['Name']]

In [19]:
# MY CODE:
astro.tail(5)

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,...,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission,Retired_Military,MissionCount,First Name,Last Name
352,David A. Wolf,1970,13,Retired,1956-08-23,"Indianapolis, IN",Male,Purdue University; Indiana University,Electrical Engineering,Medicine,...,4044,7,41.0,STS-58 (Columbia). STS-86/89 (Atlantis/Endeavo...,NaN,NaN,1,3,David,Wolf
353,Neil W. Woodward III,1970,17,Retired,1962-07-26,"Chicago, IL",Male,MIT; University of Texas-Austin; George Washin...,Physics,Physics; Business Management,...,0,0,0.0,nan,NaN,NaN,0,1,Neil,III
354,Alfred M. Worden,1970,5,Retired,1932-02-07,"Jackson, MI",Male,US Military Academy; University of Michigan,Military Science,Aeronautical & Astronautical Engineering,...,295,1,0.5,Apollo 15,NaN,NaN,1,1,Alfred,Worden
355,John W. Young,1970,2,Retired,1930-09-24,"San Francisco, CA",Male,Georgia Institute of Technology,Aeronautical Engineering,NaN,...,835,3,20.0,"Gemini 3, Gemini 10, Apollo 10, Apollo 16, STS...",NaN,NaN,1,6,John,Young
356,George D. Zamka,1970,17,Retired,1962-06-29,"Jersey City, NJ",Male,US Naval Academy; Florida Institute of Technology,Mathematics,Engineering Management,...,692,0,0.0,"STS-120 (Discovery), STS-130 (Endeavor)",NaN,NaN,1,2,George,Zamka


## CHALLENGE: Can you create a column that is the number of universities attended per astronaut?

In [20]:
# MY CODE:
astro['Number of Universities'] = [len(x.split(";")) for x in astro['Alma Mater'].astype(str)]
astro['Number of Universities'] = astro['Number of Universities'].astype('int8')
astro[['Alma Mater','Number of Universities']].head()

,Alma Mater,Number of Universities
0,University of California-Santa Barbara; Univer...,2
1,Montana State University; University of Colorado,2
2,US Military Academy; Princeton University,2
3,University of Missouri-Rolla,1
4,US Military Academy; MIT,2


### 4. Transforming our data

Now we can start re-arranging our data to get some insights!

In [21]:
# GIVEN CODE:
#Total Space Flights by Gender
grouped = astro.groupby('Gender')['Space Flights'].sum()

In [22]:
# GIVEN CODE:
#Per-person seems more equal
astro.groupby('Gender')['Space Flights'].mean()

Gender
Female    2.400000
Male      2.358306
Name: Space Flights, dtype: float64

In [23]:
# MY CODE:
# Can perform multiple aggregation in 1 step
gender_groupby_df = astro.groupby('Gender').agg({'Space Flights': ['sum','mean']})
print(gender_groupby_df)
print(gender_groupby_df.columns)

       Space Flights          
                 sum      mean
Gender                        
Female           120  2.400000
Male             724  2.358306
MultiIndex(levels=[['Space Flights'], ['sum', 'mean']],
           labels=[[0, 0], [0, 1]])


In [24]:
# MY CODE:
# Simple examples of getting just certain parts of the aggregated data
# Get just the sum of space flights
print(gender_groupby_df['Space Flights']['sum'])
print()

# Get the summary stats just for females
print(gender_groupby_df.loc['Female'])

Gender
Female    120
Male      724
Name: sum, dtype: int64

Space Flights  sum     120.0
               mean      2.4
Name: Female, dtype: float64


## CHALLENGE: Can you find the most common Undergraduate Major for Astronauts? Does that differ by Gender?

In [25]:
# MY CODE:
topx = 3

undergrad_groupby_df = astro.groupby(['Undergraduate Major'])['Name'].count().nlargest(topx)
print("Most popular Undergrad course, all genders:")
print()
print(undergrad_groupby_df)
print('\n' * 1)

undergrad_gender_groupby_df = astro.groupby(['Gender','Undergraduate Major'])['Name'].count()
genders = ['Female', 'Male']

for g in genders:
    print("Most popular Undergrad course for {}:".format(g))
    print()
    print(undergrad_gender_groupby_df.loc[g].nlargest(topx))
    print('\n' * 1)

Most popular Undergrad course, all genders:

Undergraduate Major
Physics                   35
Aerospace Engineering     33
Mechanical Engineering    30
Name: Name, dtype: int64


Most popular Undergrad course for Female:

Undergraduate Major
Chemistry                   5
Physics                     5
Aeronautical Engineering    3
Name: Name, dtype: int64


Most popular Undergrad course for Male:

Undergraduate Major
Aerospace Engineering     30
Physics                   30
Mechanical Engineering    28
Name: Name, dtype: int64




In [26]:
# GIVEN CODE:
# Pivot tables: 
pivot = astro.pivot_table(index = ['Gender','Military Branch'],values = ['Space Flight (hr)'],aggfunc = [np.mean,len])
pivot

# MY QUESTION:
#  Why is 'len' an aggregation function for pivot_table not count?????????????????????????

mean               len
                                Space Flight (hr) Space Flight (hr)
Gender Military Branch                                             
Female US Air Force                   2238.200000                 5
       US Army                         999.000000                 1
       US Naval Reserves               711.000000                 1
       US Navy                        1794.500000                 6
       nan                            1491.810811                37
Male   US Air Force                    963.519481                77
       US Air Force Reserves          1140.200000                 5
       US Army                        2758.812500                16
       US Coast Guard                 2411.500000                 2
       US Marine Corps                 651.350000                20
       US Marine Corps Reserves        506.000000                 2
       US Naval Reserves              3214.000000                 2
       US Navy                        1199.810811                74
       nan                            1183.614679               109

In [27]:
# MY CODE:
# Same output as above cell but uses pandas grouby rather than pivot_table
pivot = astro.groupby(['Gender','Military Branch'])['Space Flight (hr)'].agg({'Space Flight (hr)':['mean', 'count']})
pivot

c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


Space Flight (hr)      
                                             mean count
Gender Military Branch                                 
Female US Air Force                   2238.200000     5
       US Army                         999.000000     1
       US Naval Reserves               711.000000     1
       US Navy                        1794.500000     6
       nan                            1491.810811    37
Male   US Air Force                    963.519481    77
       US Air Force Reserves          1140.200000     5
       US Army                        2758.812500    16
       US Coast Guard                 2411.500000     2
       US Marine Corps                 651.350000    20
       US Marine Corps Reserves        506.000000     2
       US Naval Reserves              3214.000000     2
       US Navy                        1199.810811    74
       nan                            1183.614679   109

In [28]:
# GIVEN CODE:
#Sorting:
astro[['Name','MissionCount']].sort_values(by = 'MissionCount',ascending = False).head(10)

,Name,MissionCount
279,Jerry L. Ross,7
65,Franklin R. Chang-Diaz,7
111,C. Michael Foale,6
355,John W. Young,6
339,James D. Wetherbee,6
237,Story Musgrave,6
41,Curtis L. Brown Jr.,6
144,James D. Halsell Jr.,5
27,John E. Blaha,5
315,Norman E. Thagard,5


## CHALLENGE: What's the most popular first name for astronauts?

In [29]:
# MY CODE:
first_name = astro.groupby(['First Name'])['First Name'].count().nlargest(5)
first_name

First Name
Michael    16
James      15
John       13
Robert     12
William    12
Name: First Name, dtype: int64

In [30]:
# MY CODE:
firstname_pivot = astro.pivot_table(index=['First Name'], values=['Name'], aggfunc=[len])
firstname_pivot = firstname_pivot.rename(columns={'Name' : 'Count of Name'})
firstname_pivot = firstname_pivot.sort_values(by=[('len', 'Count of Name')], ascending=False)

firstname_pivot.head()

,len
,Count of Name
First Name,
Michael,16
James,15
John,13
William,12
Robert,12


## CHALLENGE: Which university has hosted the most astronauts?

In [31]:
# MY CODE:

# Why can't I use the line below? I could use it in a similar way in cell 21 (first part of Feature Engineering).
# Cell 21 was list comprehension creating a dataframe column, this example is creating a list
# unis = [x.split(";").strip() for x in astro['Alma Mater'].astype(str)]

unis = [x.split(";") for x in astro['Alma Mater'].astype(str)]
all_unis=[]
while unis:
    all_unis.extend(unis.pop(0))
all_unis = [x.strip() for x in all_unis]
# Sorting just makes it easier to eyeball the data and verify that the above logic has worked correctly
all_unis.sort()

all_unis_df = pd.DataFrame(data=all_unis, columns=['Uni'])
print(all_unis_df.head(10))

                                 Uni
0  Air Force Institute of Technology
1  Air Force Institute of Technology
2  Air Force Institute of Technology
3  Air Force Institute of Technology
4  Air Force Institute of Technology
5  Air Force Institute of Technology
6                    Amherst College
7                    Amherst College
8                  Auburn University
9                  Auburn University


In [32]:
# MY CODE:
all_unis_grouped = all_unis_df.groupby('Uni')['Uni'].count().nlargest(5)
all_unis_grouped

Uni
US Naval Academy                52
US Air Force Academy            38
MIT                             36
US Naval Postgraduate School    32
Purdue University               23
Name: Uni, dtype: int64

In [33]:
# MY CODE:
# As astronauts would likely come from a military background they would all attend a military college/university
# I think the question is more interested in civilian universities
# Therefore, re-do the analysis this time removing military colleges

# Air Force Institute of Technology is in the data with and without the 'US' prefix. I want to remove both
Military_Colleges = all_unis_df[all_unis_df['Uni'].str.contains('US|Air Force')].drop_duplicates()
Military_Colleges = list(Military_Colleges.values.flatten())

# Print out the Military colleges to eyeball check them
for c in Military_Colleges:
    print(c)

Air Force Institute of Technology
US Air Force Academy
US Air Force Institute of Technology
US Army War College
US Coast Guard Academy
US Merchant Marine Academy
US Military Academy
US Naval Academy
US Naval Postgraduate School
US Naval War College


In [34]:
# MY CODE:
all_civilian_unis_df = all_unis_df[~all_unis_df['Uni'].isin(Military_Colleges)]
print(all_civilian_unis_df.head(10))
print()
check_military_Colleges = all_civilian_unis_df[all_civilian_unis_df['Uni'].str.contains('US|Air Force')].drop_duplicates()
print("Check this has worked, the following should produce no rows")
print(check_military_Colleges)

                        Uni
6           Amherst College
7           Amherst College
8         Auburn University
9         Auburn University
10        Auburn University
11        Auburn University
12        Auburn University
13        Auburn University
14  Baldwin Wallace College
15        Baylor University

Check this has worked, the following should produce no rows
Empty DataFrame
Columns: [Uni]
Index: []


In [35]:
# MY CODE:
all_civilian_unis_grouped = all_civilian_unis_df.groupby('Uni')['Uni'].count().nlargest(10)
all_civilian_unis_grouped

Uni
MIT                                   36
Purdue University                     23
Stanford University                   22
University of Colorado                16
Georgia Institute of Technology       14
University of Texas                   13
California Institute of Technology    11
University of Washington              11
University of Southern California     10
Cornell University                     9
Name: Uni, dtype: int64